In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("data/news/results_BERT.csv")
df.head(3)

,Unnamed: 0,date,title,text,sentiment,sentiment confidence
0,0,"Thu, 21 Apr 2022 10:25:31 -0400",Mexican Football Club Tigres Now Accepts Bitco...,Tigres supporters can purchase match tickets w...,neutral,0.980093
1,1,"Thu, 21 Apr 2022 10:14:42 -0400","SFLMaven To Accept Bitcoin As Payment, Add BTC...",Luxury jewelry reseller SFLMaven announced the...,neutral,0.878789
2,2,"Thu, 21 Apr 2022 09:37:51 -0400","Bitcoin, Ethereum Technical Analysis: BTC up t...","BTC was once again trading higher, as bullish ...",positive,0.999611


In [3]:
# filter out 'uncertain' articles with sentiment confident score < 0.8
df = df[df["sentiment confidence"] > 0.8]

print(df["sentiment"].value_counts())

for index, row in df.iterrows():
  df["date"][index] = row["date"][5:-15]

df.head(2)

neutral     1735
positive    1097
negative     437
Name: sentiment, dtype: int64


/var/folders/xz/xnfjgfvn0yg69wvj2svb7crr0000gn/T/ipykernel_7407/3365207449.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["date"][index] = row["date"][5:-15]


,Unnamed: 0,date,title,text,sentiment,sentiment confidence
0,0,21 Apr 2022,Mexican Football Club Tigres Now Accepts Bitco...,Tigres supporters can purchase match tickets w...,neutral,0.980093
1,1,21 Apr 2022,"SFLMaven To Accept Bitcoin As Payment, Add BTC...",Luxury jewelry reseller SFLMaven announced the...,neutral,0.878789


In [4]:
df = df.reset_index()  # make sure indexes pair with number of rows

In [5]:
# gather negative, positive, neutral count of articles for each day
results = np.empty((0,4)) # date and sentiment counts
counts = {
  "positive": 0,
  "neutral": 0,
  "negative": 0
}
counts[df.loc[0]["sentiment"]] # initialise with the first row
for index, row in df.loc[1:].iterrows():
  if not row["date"] == df.loc[index-1]["date"]:
    results = np.vstack((results, [df.loc[index-1]["date"], counts["positive"], counts["neutral"], counts["negative"]]))
    counts["positive"] = counts["neutral"] = counts["negative"] = 0
  counts[row["sentiment"]] += 1

In [6]:
# add these columns together with the price, volume and other
df = pd.read_csv("data/BTC-USD.csv")

df = df.tail(491)
df = df.reset_index()  # make sure indexes pair with number of rows

In [7]:
results = np.flip(results, axis=0)

In [8]:
df.head(2)

,index,Date,Open,High,Low,Close,Adj Close,Volume
0,2283,2020-12-17,21308.351563,23642.660156,21234.675781,22805.162109,22805.162109,71378606374
1,2284,2020-12-18,22806.796875,23238.601563,22399.812500,23137.960938,23137.960938,40387896275


In [10]:
resultsFinal = np.empty((0, 10))
off = 0
for index, row in df.iterrows():
  temp = row.tolist()[1:]
  if not row["Date"][-2:] == results[index-off][0][:2]:
    off += 1
    temp.append(0)
    temp.append(0)
    temp.append(0)
    continue  
  temp.append(results[index-off][1])
  temp.append(results[index-off][2])
  temp.append(results[index-off][3])
  resultsFinal = np.vstack((resultsFinal, temp))

resultsFinal
  

array([['2020-12-17', '21308.351563', '23642.660156', ..., '2', '3', '0'],
       ['2020-12-18', '22806.796875', '23238.601563', ..., '3', '2', '0'],
       ['2020-12-19', '23132.865234', '24085.855469', ..., '0', '1', '0'],
       ...,
       ['2022-04-19', '40828.175781', '41672.960938', ..., '2', '8', '0'],
       ['2022-04-20', '41501.746094', '42126.300781', ..., '5', '3', '0'],
       ['2022-04-21', '41392.863281', '42878.453125', ..., '1', '1', '0']],
      dtype='<U32')

In [12]:
columns = df.columns.values.tolist()[1:]
columns.append("positive")
columns.append("neutral")
columns.append("negative")

dfFinal = pd.DataFrame(resultsFinal, columns=columns)
dfFinal.head(2)

,Date,Open,High,Low,Close,Adj Close,Volume,positive,neutral,negative
0,2020-12-17,21308.351563,23642.660156,21234.675781,22805.162109,22805.162109,71378606374,2,3,0
1,2020-12-18,22806.796875,23238.601563,22399.8125,23137.960938,23137.960938,40387896275,3,2,0


In [13]:
dfFinal.to_csv("BTC-USD-NEWS.csv")